# Joining

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120

from cut_predictor import DoubleProjectionPredictor

In [4]:
reg_xyz = DoubleProjectionPredictor()
reg_xyz.load_config('../../models/joining_xyz.pkl')
reg_xyz.load_network('../../models/best_joining_xyz_model/')
print(reg_xyz.features)

['Spanner_1', 'Spanner_2', 'Spanner_3', 'Spanner_4', 'Oberblech_MID', 'Unterblech_MID', 'Blechdicke_top', 'Niederhalterkraft_top', 'Ziehspalt_top_1.6', 'Ziehspalt_top_2.4', 'Einlegeposition_top', 'Ziehtiefe_top_50', 'Ziehtiefe_top_70', 'Rp0_top', 'Blechdicke_bot', 'Niederhalterkraft_bot', 'Ziehspalt_bot_1.6', 'Ziehspalt_bot_2.4', 'Einlegeposition_bot', 'Ziehtiefe_bot_30', 'Rp0_bot', 'pos', 'u', 'v']


In [3]:
x, y = reg_xyz.predict(
    process_parameters={
        # Joining
        'Spanner_1': -3,
        'Spanner_2': -3,
        'Spanner_3': -5,
        'Spanner_4': 3,
        'Oberblech_MID': 5,
        'Unterblech_MID': 3,

        # Top
        'Blechdicke_top': 1.48, 
        'Niederhalterkraft_top': 400, 
        'Ziehspalt_top': 2.4, 
        'Einlegeposition_top': -4, 
        'Ziehtiefe_top': 50,
        'Rp0_top': 235.160326,

        # Bottom
        'Blechdicke_bot': 1.01, 
        'Niederhalterkraft_bot': 410, 
        'Ziehspalt_bot': 2.4, 
        'Einlegeposition_bot': -5, 
        'Ziehtiefe_bot': 30,
        'Rp0_bot': 138.226960,
    },
    positions=pd.DataFrame(
        {
            "u": np.linspace(0., 1. , 100), 
            "v": np.linspace(0., 1. , 100)
        }
    ).to_numpy()
)

Spanner_1
Spanner_2
Spanner_3
Spanner_4
Oberblech_MID
Unterblech_MID
Blechdicke
Niederhalterkraft
Ziehspalt
Einlegeposition
Ziehtiefe
Rp0
Blechdicke
Niederhalterkraft
Ziehspalt
Einlegeposition
Ziehtiefe
Rp0


ValueError: in user code:

    File "/Users/vitay/Applications/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/Users/vitay/Applications/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/vitay/Applications/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/vitay/Applications/miniforge3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/Users/vitay/Applications/miniforge3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/v